In [1]:
from pathlib import Path
from keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

Using TensorFlow backend.


In [2]:
p = Path("dataset")

In [3]:
label_idx={}
labels=[]
data=[]
count=-1
for file in p.glob("*"):
    count+=1
    file_name = str(file).split("\\")[-1]
    label_idx[file_name] = count
    f = Path(file)
    for pic in f.glob("*.jpg"):
        img = image.load_img(pic,target_size=(224,224))
        img = image.img_to_array(img)
        img = preprocess_input(img)
        data.append(img)
        labels.append(count)
        
        
        
    
    
                                

In [4]:
import numpy as np
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [5]:
data.shape

(1376, 224, 224, 3)

In [6]:
labels.shape

(1376,)

In [7]:
from keras.utils import to_categorical

In [8]:
labels = to_categorical(labels)

In [9]:
x_train = data[:1100]
y_train = labels[:1100]
x_test = data[1100:]
y_test = labels[1100:]

In [10]:
aug = image.ImageDataGenerator(rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [11]:
from keras.applications import MobileNetV2
from keras.layers import *
from keras.models import Model
from keras.optimizers import Adam

In [12]:
base_model = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

C:\ProgramData\Anaconda3\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [13]:
headModel = base_model.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [14]:
model = Model(inputs=base_model.input, outputs=headModel)

In [15]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [17]:
for layer in base_model.layers:
	layer.trainable = False

In [18]:
opt = Adam(lr=1e-4, decay=1e-4 / 20)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [23]:
h = model.fit(aug.flow(x_train,y_train,batch_size=32),steps_per_epoch=len(x_train)//32,validation_data=(x_test,y_test),
              validation_steps=len(x_test)//32,epochs=3)

Epoch 1/3
34/34 [==============================] - 202s 6s/step - loss: 0.0854 - accuracy: 0.9757 - val_loss: 0.1336 - val_accuracy: 0.9493
Epoch 2/3
34/34 [==============================] - 204s 6s/step - loss: 0.0918 - accuracy: 0.9728 - val_loss: 0.0991 - val_accuracy: 0.9601
Epoch 3/3
34/34 [==============================] - 202s 6s/step - loss: 0.0898 - accuracy: 0.9644 - val_loss: 0.1295 - val_accuracy: 0.9529


In [24]:
import argparse
import os

In [26]:
model.save('FaceMaskmodel.')

usage: ipykernel_launcher.py [-h]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\my\AppData\Roaming\jupyter\runtime\kernel-b171f121-5352-4817-8ff4-66c13e44f048.json


SystemExit: 2